# Fix Time Windows - Simple Version

Updates all time windows in your ODL model to today's date.

In [1]:
import json
from datetime import datetime, timedelta

print("Imports loaded")

Imports loaded


## Configuration - CHANGE THIS

In [8]:
# Set your filename here
MODEL_FILE = "../test_model_mathias_fixed.json"

print(f"Model file: {MODEL_FILE}")

Model file: ../test_model_mathias_fixed.json


## Load Model

In [9]:
# Load the model
with open(MODEL_FILE, 'r') as f:
    model = json.load(f)

print(f"Loaded model with {len(model['data']['jobs'])} jobs")

Loaded model with 106 jobs


## Analyze Current Dates

In [14]:
# Find all dates in the model
dates = set()

for job in model['data']['jobs']:
    for stop in job['stops']:
        for tw in stop['multiTWs']:
            dt = datetime.fromisoformat(tw['openTime'])
            dates.add(dt.date())

earliest = min(dates)
today = datetime.now().date()
offset = (today - earliest).days + 1

print(f"Current dates in model: {sorted(dates)}")
print(f"Earliest date: {earliest}")
print(f"Today: {today}")
print(f"Need to shift forward: {offset} days")

Current dates in model: [datetime.date(2025, 10, 27), datetime.date(2025, 10, 28), datetime.date(2025, 10, 29)]
Earliest date: 2025-10-27
Today: 2025-10-27
Need to shift forward: 1 days


## Update Time Windows

In [15]:
# Update all time windows
count = 0

for job in model['data']['jobs']:
    for stop in job['stops']:
        for tw in stop['multiTWs']:
            # Update main time window
            old_open = datetime.fromisoformat(tw['openTime'])
            old_close = datetime.fromisoformat(tw['closeTime'])
            
            new_open = old_open + timedelta(days=offset)
            new_close = old_close + timedelta(days=offset)
            
            tw['openTime'] = new_open.isoformat()
            tw['closeTime'] = new_close.isoformat()
            
            # Update penalties
            for penalty in tw.get('penalties', []):
                old_penalty = datetime.fromisoformat(penalty['openTime'])
                new_penalty = old_penalty + timedelta(days=offset)
                penalty['openTime'] = new_penalty.isoformat()
            
            count += 1

print(f"Updated {count} time windows")

Updated 212 time windows


## Verify

In [16]:
# Check new dates
new_dates = set()

for job in model['data']['jobs']:
    for stop in job['stops']:
        for tw in stop['multiTWs']:
            dt = datetime.fromisoformat(tw['openTime'])
            new_dates.add(dt.date())

print(f"New dates in model: {sorted(new_dates)}")

# Show example
example = model['data']['jobs'][0]['stops'][0]['multiTWs'][0]
print(f"\nExample time window:")
print(f"  Open: {example['openTime']}")
print(f"  Close: {example['closeTime']}")

New dates in model: [datetime.date(2025, 10, 28), datetime.date(2025, 10, 29), datetime.date(2025, 10, 30)]

Example time window:
  Open: 2025-10-30T04:00:00
  Close: 2025-10-30T20:00:00


## Save

In [17]:
# Save the updated model
with open(MODEL_FILE, 'w') as f:
    json.dump(model, f, indent=2)

print(f"Saved updated model to: {MODEL_FILE}")
print("Done!")

Saved updated model to: ../test_model_mathias_fixed.json
Done!
